In [2]:
# context managers
    # create a context
    # execute some code 
    # clean context    

In [2]:
# __enter__
# __exit__

# use cases 
    # open a file 
    # close a file
    # Lock/release
    # change/reset
    # start/stop
    # enter/exit

# example
    # file context managers
    # decimal contexts

    # works in conjunction with a with statement

In [5]:
try: 
    10/2
except ZeroDivisionError:
    print('zero division exception occured')
finally:
    print('finally ran!')

finally ran!


In [7]:
try: 
    10/0
except ZeroDivisionError:
    print('zero division exception occured')
finally:
    print('finally ran!')

zero division exception occured
finally ran!


In [8]:
def func():
    try: 
        10/2
    except ZeroDivisionError:
        print('zero division exception occured')
    finally:
        print('finally ran!')

In [9]:
func()

finally ran!


In [10]:
try:
    print('opening file...')
    f = open('test.txt','w')
    a = 1/0
except:
    print('exception occured')
finally:
    print('closing file')
    f.close()

opening file...
exception occured
closing file


In [12]:
with open('test.txt','w') as f:
    f.writelines('this is a test')
    

In [13]:
with open('test.txt') as f:
   row = next(f)

In [14]:
row

'this is a test'

In [15]:
f.closed # context closed after reading file

True

In [16]:
# exit method
class MyContext:
    def __init__(self):
        self.obj = None
        
        
    def __enter__(self):
        print('entering context ')
        self.obj = 'the return object'
        return self.obj
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context...')
        
        if exc_type:
            print(f'**error occured:{exc_type},{exc_value}')
        return False    

In [17]:
#ctx = MyContext()
# with cts as obj:
with MyContext() as obj:
    print('inside with block')
    raise ValueError('custom message')
    

entering context 
inside with block
exiting context...
**error occured:<class 'ValueError'>,custom message


ValueError: custom message

In [19]:
ctx = MyContext()
print('create context..')
with ctx as obj:
    print('inside with block')
    raise ValueError('custom message')

create context..
entering context 
inside with block
exiting context...
**error occured:<class 'ValueError'>,custom message


ValueError: custom message

In [20]:
# exit method and supress exception
class MyContext:
    def __init__(self):
        self.obj = None
        
        
    def __enter__(self):
        print('entering context ')
        self.obj = 'the return object'
        return self.obj
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context...')
        
        if exc_type:
            print(f'**error occured:{exc_type},{exc_value}')
        return True    

In [21]:

ctx = MyContext()
print('create context..')
with ctx as obj:
    print('inside with block')
    raise ValueError('custom message')

create context..
entering context 
inside with block
exiting context...
**error occured:<class 'ValueError'>,custom message


In [22]:
class Resource:
    def __init__(self, name):
        self.name = name
        self.stats = None

In [26]:
class ResourceManager:
    def __init__(self, name):
        self.name = name
        self.resource = None
        
    def __enter__(self):
        print('entering context')
        self.resource = Resource(self.name)
        self.resource.state = 'created'
        return self.resource
    
    def __exit__(self,exc_type, exc_value, exc_tb):
        print('exiting context')
        self.resource.state = 'destroyed'
        if exc_type:
            print('error occured')
        return False    

In [28]:
with ResourceManager('spam') as res:
    print(f'{res.name} = {res.state}')
print('file closed..')    
print(f'{res.name} = {res.state}')          

entering context
spam = created
exiting context
file closed..
spam = destroyed


In [29]:
class File:
    
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
    
    def __enter__(self):
        print('opening file..')
        self.file = open(self.name, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('closing file')
        self.file.close()
        return False

In [30]:
with File('test.txt','w') as f:
    f.write('this is a later parrot')

opening file..
closing file


In [32]:
with File('test.txt','r') as f:
    print(f.readlines())

opening file..
['this is a later parrot']
closing file


In [33]:
class File:
    
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
    
    def __enter__(self):
        print('opening file..')
        self.file = open(self.name, self.mode)
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('closing file')
        self.file.close()
        return False

In [35]:
with File('test.txt','r') as f_ctx:
    print(next(f_ctx.file))
    print(f_ctx.name)
    print(f_ctx.mode)

opening file..
this is a later parrot
test.txt
r
closing file


# Caveat with lazy iterators

In [37]:
import csv

def read_data():
    with open('park_ticket.csv') as f:
        return csv.reader(f, delimiter = ',', quotechar = '"')

In [ ]:
reader = read_data()

In [ ]:
from row in reader: # this will error out as file will close before the iteration completes
    print(row)

In [ ]:
# RESOLVE 1
import csv

def read_data():
    with open('park_ticket.csv') as f:
         yield from csv.reader(f, delimiter = ',', quotechar = '"')


In [ ]:
# RESOLVE 2 

import csv

def read_data():
    with open('park_ticket.csv') as f:
        return list(csv.reader(f, delimiter = ',', quotechar = '"'))

# Not just a ContextManager

In [39]:
# context manager class can be used for other functions i.e. not just context functions

In [40]:
with open('test.txt', 'w') as f:
    f.writelines('this is a test')

In [41]:
f = open('test.txt')
print(f.readlines())
f.close()

['this is a test']


In [42]:
class DataIterator:
    def __init__(self, fname):
        self._fname = fname 
        self._f = None
        
    def __iter__(self):
        return self
    
    def __next__(self):
        row = next(self._f)
        return row.strip('\n').split(',')
    
    def __enter__(self):
        self._f = open(self._fname)
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        if not self._f.closed:
            self._f.close()
        return False    

In [43]:
with DataIterator('test.txt') as data:
    for row in data:
        print(row)

['this is a test']


# Additional Usecases

In [44]:
import decimal 

In [45]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [47]:
decimal.getcontext().prec = 14

In [48]:
decimal.getcontext()

Context(prec=14, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [49]:
decimal.getcontext().prec = 28

In [50]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [51]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 4
print(decimal.Decimal(1)/decimal.Decimal(3))
decimal.getcontext().prec = old_prec
print(decimal.Decimal(1)/decimal.Decimal(3))

0.3333
0.3333333333333333333333333333


In [52]:
class Precision:
    def __init__(self,prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec
        
        
    def __enter__(self):
        decimal.getcontext().prec = self.prec
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self.current_prec
        return False
    
    
    

In [53]:
with Precision(3):
    print(decimal.Decimal(1)/decimal.Decimal(3))
print(decimal.Decimal(1)/decimal.Decimal(3))    

0.333
0.3333333333333333333333333333


In [54]:
with decimal.localcontext() as cts:
    cts.prec = 3
    print(decimal.Decimal(1)/decimal.Decimal(3))
print(decimal.Decimal(1)/decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In [55]:
from time import perf_counter, sleep


In [57]:
class Timer:
    def __init__(self):
        self.elapsed = 0 
        
    def __enter__(self):
        self.start = perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop = self.start
        return False

In [59]:
with Timer() as timer:
    sleep(1)
print(timer.elapsed)

7568.791847151


In [60]:
import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout
        
    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stdout = self._file
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._current_stdout
        self._file.close()
        return False

In [61]:
with OutToFile('test.txt'):
    print('Line 1')
    print('Line 2')

In [63]:
class Tag:
    def __init__(self, tag):
        self._tag = tag
        
    def __enter__(self):
        print(f'<{self._tag}>', end = '')
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f'</{self._tag}>', end = '')
        return False

In [64]:
with Tag('p'):
    print('some ', end= '')
    with Tag('b'):
        print('bold', end = '')
    print('text ', end= '')    

<p>some <b>bold</b>text </p>

In [67]:
class ListMaker:
    def __init__(self, title, prefix = '- ', indent = 3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)
        
    def __enter__(self):
        self._current_indent += self._indent
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent
        return False
    
    def print(self, arg):
        s = ' '*self._current_indent + self._prefix + str(arg)
        print(s)

In [68]:
lm = ListMaker('Items')

Items


In [69]:
with ListMaker('Items') as lm:
    lm.print('Item 1')
    lm.print('Item 2')

Items
   - Item 1
   - Item 2


In [72]:
with ListMaker('Items') as lm:
    lm.print('Item 1')
    with lm:
        lm.print('subitem 1a')
        lm.print('subitem 1b')
    lm.print('Item 2')
    with lm:
        lm.print('subitem 2a')
        lm.print('subitem 2b')

Items
   - Item 1
      - subitem 1a
      - subitem 1b
   - Item 2
      - subitem 2a
      - subitem 2b


In [78]:
with OutToFile('mylist.txt'):
    with ListMaker('Items') as lm:
        lm.print('Item 1')
        with lm:
            lm.print('subitem 1a')
            lm.print('subitem 1b')
        lm.print('Item 2')
        with lm:
            lm.print('subitem 2a')
            lm.print('subitem 2b')

In [79]:
print('test')

test


In [81]:
with open('mylist.txt') as f:
    for row in f:
        print(row, end = '')

Items
   - Item 1
      - subitem 1a
      - subitem 1b
   - Item 2
      - subitem 2a
      - subitem 2b


# Generators and context managers